A bit of code to extract all the individual cropped images from all the cameras and copy them to a separate folder. The cropped images are put into folders according to the taxonomic prediction from BioClip. Additionally, a folder containing all crops is also created. 

Expects the data structure that results from the main pipeline. Results from each camera should be placed in new folder that is contained within the individual camera folder in 'all_cameras_folder' (e.g DIOPSIS-383_analyzed in 383_Weiss_low in 'all_cameras_folder').

Activate test mode to check whether everything runs as expected. In test_mode, only the first crop of each camera is copied so that mistakes do not directly destroy all your data.

In [1]:
import os
import csv
import shutil
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
all_cameras_folder = r'N:\alpenv\GOES\Projects\Diopsis Cameras\RESULTS_2024\Images_RAW_and_analyzed'
output_folder = r"N:\alpenv\GOES\Projects\Diopsis Cameras\RESULTS_2024\all_crops"

test_mode = False

all_crops_path = os.path.join(output_folder, "all_crops")
os.makedirs(all_crops_path, exist_ok=True)
all_cameras_folders = os.listdir(all_cameras_folder)
all_cameras_folders

['382_Mon_mid',
 '383_Weiss_low',
 '384_Weiss_mid',
 '385_Weiss_up',
 '387_Jatz_mid',
 '388_Jatz_up',
 '389_Mon_low',
 '391_Mon_up',
 '421_Jatz_low']

In [5]:
order_csv = False
os.makedirs(all_crops_path, exist_ok=True)
for camera_folder in all_cameras_folders:
    folder_path = os.path.join(all_cameras_folder, camera_folder)
    folders = os.listdir(folder_path)
    folder_name = ''
    for path in folders:
        if path.endswith('_analyzed'):
            folder_name = path
            break
    analyzed_path = os.path.join(folder_path, folder_name)
    try:
        csv1 = pd.read_csv(os.path.join(analyzed_path, "results_order_classification.csv"))
        order_csv = True
    except FileNotFoundError:
        print(f"CSV for special order classification not found. Resorting to basic results csv.")
        try:
            csv = pd.read_csv(os.path.join(analyzed_path, "all_analysis_results.csv"))
        except FileNotFoundError:
            print(f"No results csv found in {camera_folder}")
            continue
    for index, row in tqdm(csv1.iterrows()):
        if order_csv:
            filepath = row['file_name']
            filename = os.path.basename(filepath)
        else:
            filename = row['Filename']
        day_folder = filename[:8]
        order = row['order']
        pred_class = row['class']
        destination_folder = os.path.join(output_folder, pred_class, order)
        img_filepath = os.path.join(analyzed_path, "photos", day_folder, "cropped", filename)
        if not os.path.exists(img_filepath):
            print(f"Source file {img_filepath} cannot be found. Skipping this file")
            continue
        os.makedirs(destination_folder, exist_ok=True)
        destination_path = os.path.join(destination_folder, filename)
        try:
            shutil.copy2(img_filepath, destination_path)
            shutil.copy2(img_filepath, all_crops_path)
            #print(f"Copied {img_filepath} to {destination_path}")
        except Exception as e:
            print(f"Error copying {img_filepath} to {destination_path}: {e}")
        if test_mode:
            break
    print(f"{camera_folder} done.")

0it [00:00, ?it/s]

382_Mon_mid done.


0it [00:00, ?it/s]

383_Weiss_low done.


0it [00:00, ?it/s]

...

389_Mon_low done.


0it [00:00, ?it/s]

391_Mon_up done.


0it [00:00, ?it/s]

421_Jatz_low done.
